In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from BCBio import GFF
import pathlib # for checking the existance of a file
import os
path2Works = '/mnt/d/WORKs'

In [2]:
# Clear the log file if exist
def clear_log(logfile):
    try:
        pathlib.Path(os.path.dirname(logfile)).mkdir(parents=True,exist_ok=True)
        if pathlib.Path(logfile).is_file():
            log_handle = open(logfile, 'w')
            log_handle.close()
            write_log('Previous log cleaned\n')
    except:
        pass
    
def write_log(logstr, end = None):
    try:
        log_handle = open(logfile, 'a')
        print(logstr, end = end, file = log_handle)
        print(logstr, end = end)
        log_handle.close()
    except:
        pass

In [3]:

def embl_to_gb(scnum):
    '''Convert entire scaffold - one parse instance from SeqIO.parse'''
# Read the file
    embl_file = os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/sc{str(scnum).zfill(2)}.embl")
    output_file = os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/sc{str(scnum).zfill(2)}.gb")
    sample = SeqIO.read(embl_file,"embl")
    write_log(f'{embl_file}\n has {len(sample.features)} features.\nConverting to genbank file...\n')

# Parse embl file to gb file with correct annotation:
    write_log('################# Converting features #################\n')
    for feat_num, feat in enumerate(sample.features):
        if 'mol_type' in feat.qualifiers:
            continue #Ignore:
                     #FT   source          1..441420
                     #FT                   /mol_type="genomic DNA"
        write_log(f'No.{feat_num}:')
        if feat.qualifiers['gene'][0] != 'unknown': # if this feature have been assigned to a protein name
            try:
                feat.qualifiers['inference'] = [] # Set up 'inference' field to convert convertable information to genbank official format, 'inference' qualifier is more accurate in describing the feature. Here for QL109 we have blastp_match and domain match, so it is a list.
                for note in feat.qualifiers['note']:
                    if note.startswith('ORF '): # Every feature has a ORF number in QL109 embl files, write the information on screen, together with protein name and similarity value
                        write_log(note, end = ': ')
                        write_log(feat.qualifiers['gene'][0] + '\n\t' + feat.qualifiers['similarity'][0])
                        ORF_num = note.split(' ')[2]
                        feat.qualifiers['note'].remove(note) # I will put the information in valid qualifier 'protein_id' and 'locus_tag'
                    elif note.startswith('homologous to '):
                        refseq_id = note.split(' ')[2].split('|')[1].split('_')[1]
                        feat.qualifiers['inference'].append(f'similar to AA sequence:UniProtKB:{refseq_id}')
                feat.qualifiers['note'].append(f'blastp_match: {"; ".join(feat.qualifiers["blastp_match"])}, similarity: {"; ".join(feat.qualifiers["similarity"])}') # Put redundant information in another note
                del feat.qualifiers["blastp_match"], feat.qualifiers["similarity"]
                
                if 'domain' in feat.qualifiers:
                    pfams = []
                    for domain in feat.qualifiers['domain']:
                        pfams.append(domain.split(' ')[1])
                        feat.qualifiers['note'].append(f'domain: {domain}') # Put redundant information in another note
                    feat.qualifiers['inference'].append(f'protein motif:PFAM:{",".join(pfams)}')
                    del feat.qualifiers['domain']
            except:
                write_log('Something wrong?')
                raise
        else: # if the gene has no homologe, calculate orf number here.
            ORF_num = str(feat_num).zfill(4)
            write_log(f'ORF number {ORF_num} on the contig: Unknown protein')
        
        feat.qualifiers['codon_start'] = 1
        feat.qualifiers['transl_table'] = 11
        locus_tag = f'QL109_SC{str(scnum).zfill(2)}ORF{ORF_num}'
        protein_id = locus_tag[6:]
        feat.qualifiers['ID'] = protein_id
        feat.qualifiers['Name'] = feat.qualifiers['gene'][0]
        del feat.qualifiers['gene']
        feat.qualifiers['locus_tag'] = locus_tag
        strand = f'{feat.location.strand:+}'[0]
        write_log(f'\tStrand: {strand}\n')
        # if feat_num == 2:
            # break
    # Rename the sample, if the name is too long, shorten it
    LOCUS = f"QL109_SC{str(scnum).zfill(2)}"
    sample.name = LOCUS
    
    if pathlib.Path(output_file).is_file():
        caution = 'Caution file overwrote:\n'
    else:
        caution = ''
    write_log(f'Writing genbank file...\n')
    write_flag = SeqIO.write(sample, output_file, "genbank")
    # Show ending information
    if write_flag == 1:
        write_log(f"\n########## Output files ##########\n\n{caution}{output_file}\n{logfile}\n")
    else:
        print(f'Something wrong in writing file to\n{output_file}')


In [4]:
for i in range(1,14):
    logfile = os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/log/sc{str(i).zfill(2)}_embl_to_gb.log")
    clear_log(logfile)
    embl_to_gb(i)

Previous log cleaned

/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/sc01.embl
 has 405 features.
Converting to genbank file...

################# Converting features #################

No.1:
ORF number 0001 on the contig: Siderophore 2 3-dihydroxybenzoate-glycine-threonine trimeric ester bacillibactin synthetase (Fragment)
	identity of 55.97 percent
	Strand: -

No.2:
ORF number 0002 on the contig: Amino acid adenylation domain protein
	identity of 49.34 percent
	Strand: -

No.3:
ORF number 0003 on the contig: MbtH domain-containing protein
	identity of 74.19 percent
	Strand: -

No.4:
ORF number 0004 on the contig: Ferredoxin-dependent glutamate synthase
	identity of 67.32 percent
	Strand: -

No.5:
ORF number 0005 on the contig: Asparaginyl-tRNA synthetase-like
	identity of 70.10 percent
	Strand: -

No.6:
ORF number 0006 on the contig: Methylaspartate mutase
	identity of 58.81 percent
	Strand: -

No.7:
ORF number 0007 on the contig: Glutamate mutase subunit A
	identity of 56.12 per

No.81:
ORF number 0081 on the contig: Arginine deiminase
	identity of 34.86 percent
	Strand: -

No.82:
ORF number 0082 on the contig: Arginine deiminase 1
	identity of 28.80 percent
	Strand: -

No.83:
ORF number 0083 on the contig: Alkaline d-peptidase. serine peptidase. merops family s12
	identity of 63.41 percent
	Strand: +

No.84:
ORF number 0084 on the contig: Uncharacterized protein
	identity of 54.55 percent
	Strand: +

No.85:
ORF number 0085 on the contig: Predicted protein
	identity of 74.00 percent
	Strand: -

No.86:
ORF number 0086 on the contig: ArsR family transcriptional regulator
	identity of 85.44 percent
	Strand: +

No.87:
ORF number 0087 on the contig: Vegetative cell wall protein gp1 (Hydroxyproline-rich glycoprotein 1) (Fragment)
	identity of 51.40 percent
	Strand: +

No.88:
ORF number 0088 on the contig: Transcriptional regulator  GntR family with aminotransferase domain
	identity of 58.37 percent
	Strand: -

No.89:
ORF number 0089 on the contig: 3 5-cyclic-nucleoti

ORF number 0159 on the contig: Transcriptional regulator
	identity of 84.78 percent
	Strand: +

No.160:
ORF number 0160 on the contig: GCN5-related N-acetyltransferase
	identity of 88.64 percent
	Strand: +

No.161:
ORF number 0161 on the contig: Predicted protein
	identity of 54.29 percent
	Strand: +

No.162:
ORF number 0162 on the contig: Putative uncharacterized protein
	identity of 81.08 percent
	Strand: -

No.163:
ORF number 0163 on the contig: TPR repeat
	identity of 40.81 percent
	Strand: -

No.164:
ORF number 0164 on the contig: Predicted protein
	identity of 39.83 percent
	Strand: -

No.165:
ORF number 0165 on the contig: Putative uncharacterized protein
	identity of 29.14 percent
	Strand: +

No.166:
ORF number 0166 on the contig: Polyketide synthase
	identity of 39.66 percent
	Strand: +

No.167:
ORF number 0167 on the contig: Serine hydroxymethyltransferase
	identity of 34.09 percent
	Strand: +

No.168:
ORF number 0168 on the contig: Glyoxalase/bleomycin resistance protein/dio

	Strand: +

No.233:
ORF number 0233 on the contig: Transcriptional regulator
	identity of 81.32 percent
	Strand: -

No.234:
ORF number 0234 on the contig: DJ-1/PfpI-family transcriptional regulator
	identity of 79.57 percent
	Strand: +

No.235:
ORF number 0235 on the contig: Putative uncharacterized protein (Fragment)
	identity of 31.52 percent
	Strand: -

No.236:
ORF number 0236 on the contig: Exodeoxyribonuclease III
	identity of 23.85 percent
	Strand: -

No.237:
ORF number 0237 on the contig: Putative two-component system histidine kinase
	identity of 45.74 percent
	Strand: +

No.238:
ORF number 0238 on the contig: Putative two-component system response regulator
	identity of 51.90 percent
	Strand: +

No.239:
ORF number 0239 on the contig: Transcriptional regulator  LuxR family
	identity of 45.26 percent
	Strand: +

No.240:
ORF number 0240 on the contig: ABC transporter related
	identity of 26.72 percent
	Strand: -

No.241:
ORF number 0241 on the contig: Serine/threonine protein kin

No.314:
ORF number 0314 on the contig: Transposase
	identity of 71.11 percent
	Strand: -

No.315:
ORF number 0315 on the contig: Predicted protein
	identity of 37.89 percent
	Strand: +

No.316:
ORF number 0316 on the contig: Putative membrane protein
	identity of 24.36 percent
	Strand: -

No.317:
ORF number 0317 on the contig: CAAX amino terminal protease family protein
	identity of 39.24 percent
	Strand: -

No.318:
ORF number 0318 on the contig: Putative uncharacterized protein
	identity of 30.68 percent
	Strand: -

No.319:
ORF number 0319 on the contig: Methyltransferase type 11
	identity of 47.89 percent
	Strand: -

No.320:
ORF number 0320 on the contig: Putative uncharacterized protein
	identity of 36.84 percent
	Strand: -

No.321:
ORF number 0321 on the contig: Uncharacterized protein
	identity of 51.85 percent
	Strand: -

No.322:
ORF number 0322 on the contig: Glycosyl transferase  family protein 2
	identity of 37.04 percent
	Strand: -

No.323:
ORF number 0323 on the contig: HAD-

...

ORF number 0199 on the contig: Extracellular solute-binding protein
	identity of 36.92 percent
	Strand: -

No.200:
ORF number 0200 on the contig: Predicted protein
	identity of 57.02 percent
	Strand: -

No.201:
ORF number 0201 on the contig: Predicted protein
	identity of 30.84 percent
	Strand: -

No.202:
ORF number 0202 on the contig: Uncharacterized protein
	identity of 62.28 percent
	Strand: +

No.203:
ORF number 0203 on the contig: Nitrilase/cyanide hydratase and apolipoprotein N-acyltransferase
	identity of 69.70 percent
	Strand: -

No.204:
ORF number 0204 on the contig: Putative uncharacterized protein
	identity of 32.46 percent
	Strand: -

No.205:
ORF number 0205 on the contig: Unknown protein
	Strand: -

No.206:
ORF number 0206 on the contig: ATP-dependent chaperone ClpB
	identity of 84.57 percent
	Strand: +

No.207:
ORF number 0207 on the contig: Uncharacterized protein
	identity of 76.65 percent
	Strand: +

No.208:
ORF number 0208 on the contig: Putative membrane protein
	ide

ORF number 0005 on the contig: Predicted protein
	identity of 100.00 percent
	Strand: +

No.6:
ORF number 0006 on the contig: NAD-dependent epimerase/dehydratase
	identity of 85.17 percent
	Strand: +

No.7:
ORF number 0007 on the contig: Phospholipid/glycerol acyltransferase
	identity of 78.41 percent
	Strand: +

No.8:
ORF number 0008 on the contig: Putative membrane protein
	identity of 63.92 percent
	Strand: -

No.9:
ORF number 0009 on the contig: AdsA-like sigma factor
	identity of 98.31 percent
	Strand: -

No.10:
ORF number 0010 on the contig: Putative non-ribosomal peptide synthetase
	identity of 41.86 percent
	Strand: +

No.11:
ORF number 0011 on the contig: 3-phosphoserine phosphatase
	identity of 84.33 percent
	Strand: -

No.12:
ORF number 0012 on the contig: Putative redoxin
	identity of 83.16 percent
	Strand: -

No.13:
ORF number 0013 on the contig: Redox-sensing transcriptional repressor rex
	identity of 84.02 percent
	Strand: +

No.14:
ORF number 0014 on the contig: Glutamy

	Strand: +

No.87:
ORF number 0087 on the contig: ABC transporter membrane protein
	identity of 77.67 percent
	Strand: +

No.88:
ORF number 0088 on the contig: Aminopeptidase S
	identity of 72.04 percent
	Strand: -

No.89:
ORF number 0089 on the contig: RarD protein  DMT superfamily transporter
	identity of 67.48 percent
	Strand: -

No.90:
ORF number 0090 on the contig: Integral membrane protein
	identity of 73.48 percent
	Strand: -

No.91:
ORF number 0091 on the contig: 2-oxoglutarate oxidoreductase  beta subunit
	identity of 83.18 percent
	Strand: -

Writing genbank file...


########## Output files ##########

/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/gb_file_new/sc07.gb
/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/gb_file_new/log/sc07_embl_to_gb.log

/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/sc08.embl
 has 154 features.
Converting to genbank file...

################# Converting features #################

No.1:
ORF number 0001 on the contig: Phosphoenolpyru

No.70:
ORF number 0070 on the contig: Acyl-CoA dehydrogenase
	identity of 83.59 percent
	Strand: -

No.71:
ORF number 0071 on the contig: Chitinase 63
	identity of 78.07 percent
	Strand: +

No.72:
ORF number 0072 on the contig: Lipase class 2
	identity of 62.05 percent
	Strand: -

No.73:
ORF number 0073 on the contig: Dihydroxyacetone kinase  L subunit
	identity of 61.58 percent
	Strand: +

No.74:
ORF number 0074 on the contig: Magnesium or manganese-dependent protein phosphatase
	identity of 38.64 percent
	Strand: +

No.75:
ORF number 0075 on the contig: LmbE family protein
	identity of 74.07 percent
	Strand: -

No.76:
ORF number 0076 on the contig: Lipoprotein
	identity of 68.91 percent
	Strand: +

No.77:
ORF number 0077 on the contig: Citrate-dependent iron (III) transporter
	identity of 79.28 percent
	Strand: +

No.78:
ORF number 0078 on the contig: Putative Fe uptake ABC transporter ATP-binding protein
	identity of 76.45 percent
	Strand: +

No.79:
ORF number 0079 on the contig: Un

	Strand: -

No.146:
ORF number 0146 on the contig: Cytochrome P450
	identity of 55.81 percent
	Strand: -

No.147:
ORF number 0147 on the contig: Transcriptional regulator  DeoR family
	identity of 92.88 percent
	Strand: -

No.148:
ORF number 0148 on the contig: Ribulose-phosphate 3-epimerase
	identity of 89.91 percent
	Strand: -

No.149:
ORF number 0149 on the contig: Putative RNA-binding Sun protein
	identity of 85.20 percent
	Strand: -

No.150:
ORF number 0150 on the contig: Methionyl-tRNA formyltransferase
	identity of 86.50 percent
	Strand: -

No.151:
ORF number 0151 on the contig: Uncharacterized protein
	identity of 98.28 percent
	Strand: +

No.152:
ORF number 0152 on the contig: Putative primosomal protein
	identity of 75.00 percent
	Strand: -

No.153:
ORF number 0153 on the contig: PokSAMS (Fragment)
	identity of 80.26 percent
	Strand: -

Writing genbank file...


########## Output files ##########

/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/gb_file_new/sc08.gb
/mnt/d/W

In [8]:
allInOne = SeqIO.read(os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/sc01.gb"),'genbank')
for i in range (2,14):
    pathGbFile = os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/sc{str(i).zfill(2)}.gb")
    allInOne += SeqIO.read(pathGbFile, 'genbank')
allInOne.name = 'QL109_allContigs'
allInOne.id = 'QL109_allContigs'

with open(os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/QL109_allInOne.gb"), 'w') as targetGenbank:
    SeqIO.write(allInOne, targetGenbank, 'genbank')

In [5]:
allIn13 = list()
allIn13.append(SeqIO.read(os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/sc01.gb"),'genbank'))
for i in range (2,14):
    pathGbFile = os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/sc{str(i).zfill(2)}.gb")
    allIn13.append(SeqIO.read(pathGbFile, 'genbank'))

with open(os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/gb_file_new/QL109_allIn13.gb"), 'w') as targetGenbank:
    SeqIO.write(allIn13, targetGenbank, 'genbank')

In [3]:
with open(os.path.join(path2Works, f"Resources/Resource_QL109/Genome_QL109/QL109_allInOne.gb"), 'r') as source,\
open('/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/gb_file_new/QL109_allInOne.gb.Notworking.gff', 'w') as targetGff,\
open('/mnt/d/WORKs/Resources/Resource_QL109/Genome_QL109/gb_file_new/QL109_allInOne.gb.fasta', 'w') as targetFasta:
    sourceObj = list(SeqIO.parse(source, 'genbank'))
    GFF.write(sourceObj, targetGff)
    SeqIO.write(sourceObj, targetFasta, 'fasta')
